##*Modelado de datos para las funciones de ML*

###Importamos Librerias

In [1]:
import pandas as pd
import numpy as np

import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
import operator

import fastparquet as fp 
import pyarrow as pa
import pyarrow.parquet as pq

###Modelo de recomendación Item_Item

Un modelo de recomendación item-item basado en la similitud del coseno es un tipo de sistema de recomendación que sugiere elementos similares a aquellos que un usuario ha mostrado interés previamente o ha interactuado. Este enfoque se fundamenta en la premisa de que si a un usuario le gustó un cierto ítem, es probable que también le gusten ítems similares. El perfil de un ítem se representa comúnmente como un vector que contiene información relevante sobre ese ítem. Por ejemplo, en un sistema de recomendación de películas, el perfil de una película podría incluir datos como el género, el director, el elenco de actores, las etiquetas asociadas, entre otros. El cálculo de la similitud del coseno se emplea para evaluar la similitud entre los perfiles de los ítems y se basa en el ángulo entre los vectores de características de los ítems. Cuanto menor sea el ángulo entre los vectores, mayor será la similitud del coseno y más parecidos serán los ítems.

###Se inicia con la creación de dummies , para luego crear la matriz necesaria para el analisis de similitud de coseno

In [2]:
modelo_item = pd.read_parquet("modelo_item.parquet")
modelo_item.head(3)

,item_id,app_name,genres
0,761140,Lost Summoner Kitty,Action
1,643980,Ironbound,Free to Play
2,670290,Real Pool 3D - Poolians,Casual


###Generamos un DataFrame que incluye en sus columnas los géneros que pueden ser combinados con los nombres de los juegos en cada una de las filas.

In [3]:
#Creamos los dummies
modelo_item = pd.get_dummies(modelo_item, columns=["genres"], prefix="")

modelo_item = modelo_item.groupby(["item_id","app_name"]).sum().reset_index()

modelo_item.head(4)

,item_id,app_name,_Accounting,_Action,_Adventure,_Animation &amp; Modeling,_Audio Production,_Casual,_Design &amp; Illustration,_Early Access,...,_Photo Editing,_RPG,_Racing,_Simulation,_Software Training,_Sports,_Strategy,_Utilities,_Video Production,_Web Publishing
0,10,Counter-Strike,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20,Team Fortress Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,Day of Defeat,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,40,Deathmatch Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


###Calcular la similitud del coseno

En un modelo de recomendación item-item basado en similitud del coseno, se calcula la similitud del coseno entre todos los pares posibles de ítems en función de sus perfiles. La similitud del coseno mide el ángulo entre dos vectores en un espacio multidimensional y varía de -1 (completamente opuestos) a 1 (idénticos). Cuanto mayor sea la similitud del coseno entre dos ítems, más parecidos se considerarán. Este enfoque permite identificar ítems que comparten características similares, lo que facilita la recomendación de ítems relevantes para los usuarios.

In [4]:
similitudes = cosine_similarity(modelo_item.iloc[:,3:])

In [5]:
similitudes.shape

(28899, 28899)

In [6]:
print(similitudes)

[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]


In [7]:
def recomendacion_juego(id):
    
    id = int(id)
    # Filtramos el juego e igualarlo a  su ID
    juego_seleccionado = modelo_item[modelo_item['item_id'] == id]
    # devolvemos error en caso de vacio
    if juego_seleccionado.empty:
        return "El juego con el ID especificado no existe en la base de datos."
    
    # Calculamos la matriz de similitud coseno
    #similitudes = cosine_similarity(modelo_item.iloc[:,3:])
    
    # Calculamos la similitud del juego que se ingresa con otros juegos del dataframe
    similarity_scores = similitudes[modelo_item[modelo_item['item_id'] == id].index[0]]
    
    # Calculamos los índices de los juegos más similares (excluyendo el juego de entrada)
    indices_juegos_similares = similarity_scores.argsort()[::-1][1:6]
    
    # Obtenemos los nombres de los juegos 5 recomendados
    juegos_recomendados = modelo_item.iloc[indices_juegos_similares]['app_name']
    
    return juegos_recomendados

In [8]:
recomendacion_juego(2028103)

13234                    Castle Chaos
13436    The Odyssey: Winds of Athena
13435                       Mars 2030
13434                        Westboro
13433                           CLASH
Name: app_name, dtype: object

In [9]:
modelo_item

,item_id,app_name,_Accounting,_Action,_Adventure,_Animation &amp; Modeling,_Audio Production,_Casual,_Design &amp; Illustration,_Early Access,...,_Photo Editing,_RPG,_Racing,_Simulation,_Software Training,_Sports,_Strategy,_Utilities,_Video Production,_Web Publishing
0,10,Counter-Strike,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20,Team Fortress Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,Day of Defeat,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,40,Deathmatch Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,50,Half-Life: Opposing Force,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28894,2028055,Tom Clancy's Ghost Recon Future Soldier - Seas...,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28895,2028056,Worms Revolution Season Pass,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
28896,2028062,Call of Duty®: Black Ops II Season Pass,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28897,2028103,Assassin’s Creed® III Season Pass,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##Dividimos el dataframe por la mitad a fin de lograr montar el modelo en render

In [11]:
# Revisamos la cantidad de filas
cant_filas= len(modelo_item)

# Calculamos la mitad
mitad_filas= cant_filas // 10
 
# Seleccionamos la mitad superior
modelo_train= modelo_item.iloc[:mitad_filas]

In [12]:
modelo_train.shape

(2889, 25)

In [13]:
similitudes_train = cosine_similarity(modelo_train.iloc[:,3:])

###Almacenamos el dataframe acotado

In [19]:
modelo_train.to_parquet("modelo_train.parquet")

Se crea una nueva función diseñada para explorar exclusivamente las similitudes con la fila del juego especificada como parámetro. Esto se realiza con el propósito de facilitar la ejecución del archivo mediante el proceso de renderizado. La función tomará como entrada el juego específico y calculará las similitudes con todos los demás juegos en función de sus perfiles. Esto proporciona una manera eficiente de obtener recomendaciones para un juego dado sin tener que calcular todas las similitudes cada vez.

In [15]:
def encontrar_juegos_similares(id):
    # Encontramos el índice del juego ingresado por ID
    juego_indice = modelo_train.index[modelo_train['item_id'] == id].tolist()[0]
    
    # Extraemos las características del juego ingresado
    juego_caracteristicas = modelo_train.iloc[juego_indice, 3:].values.reshape(1, -1)
    
    # Calculamos la similitud coseno entre el juego ingresado y todos los demás juegos
    similitudes_railway = cosine_similarity(modelo_train.iloc[:, 3:], juego_caracteristicas)
    
    # Obtenemos los índices de los juegos más similares (excluyendo el juego de entrada)
    indices_juegos_similares = similitudes_train.argsort(axis=0)[::-1][1:6]
    indices_juegos_similares = indices_juegos_similares.flatten()[1:]
    
    # Obtenemos los juegos más similares en función de los índices
    juegos_similares = modelo_train.iloc[indices_juegos_similares]['app_name']
    
    return juegos_similares

In [16]:
encontrar_juegos_similares(3310)

1337    Warhammer 40,000: Space Marine - Traitor Legio...
1337    Warhammer 40,000: Space Marine - Traitor Legio...
1337    Warhammer 40,000: Space Marine - Traitor Legio...
1337    Warhammer 40,000: Space Marine - Traitor Legio...
1337    Warhammer 40,000: Space Marine - Traitor Legio...
                              ...                        
1869                             Naval War: Arctic Circle
2628                            Atooms to Moolecules Demo
2628                            Atooms to Moolecules Demo
1368    Warhammer 40,000: Space Marine - Death Guard C...
1368    Warhammer 40,000: Space Marine - Death Guard C...
Name: app_name, Length: 14444, dtype: object

###Creamos una función para obtener recomendaciones

In [17]:
def recomendacion_juego(id: int):
    
    # Verificamos si el juego con game_id existe en df_games
    game = modelo_train[modelo_train['item_id'] == id]

    if game.empty:
        return("El juego '{id}' no posee registros.")
    
    # Obtenemos el índice del juego dado
    idx = game.index[0]

    # Tomamos una muestra aleatoria del DataFrame df_games
    sample_size = 2000  # Definimos el tamaño de la muestra (ajusta según sea necesario)
    df_sample = modelo_train.sample(n=sample_size, random_state=42)  # Ajustamos la semilla aleatoria según sea necesario

    # Calculamos la similitud de contenido solo para el juego dado y la muestra
    sim_scores = cosine_similarity([modelo_train.iloc[idx, 3:]], df_sample.iloc[:, 3:])

    # Obtenemos las puntuaciones de similitud del juego dado con otros juegos
    sim_scores = sim_scores[0]

    # Ordenamos los juegos por similitud en orden descendente
    similar_games = [(i, sim_scores[i]) for i in range(len(sim_scores)) if i != idx]
    similar_games = sorted(similar_games, key=lambda x: x[1], reverse=True)

    # Obtenemos los 5 juegos más similares
    similar_game_indices = [i[0] for i in similar_games[:5]]

    # Listamos los juegos similares (solo nombres)
    similar_game_names = df_sample['app_name'].iloc[similar_game_indices].tolist()

    return {"similar_games": similar_game_names}

In [18]:
recomendacion_juego(10)

{'similar_games': ['Tomb Raider: Shanty Town',
  'Magicka: Holiday Spirit Item Pack',
  'Nuclear Dawn',
  'Sam & Max 104: Abe Lincoln Must Die!',
  'Shadowgrounds Survivor']}